In [182]:
import requests
import json
from bs4 import BeautifulSoup

import re #for stripping out characters
import regex #for string matching
from tqdm import tqdm


In [41]:
#textual updating code
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [42]:
def proposal_process(proplist,requestpath):
    '''takes a list with proposal info and processes some info into a df '''
    #we can use values in these keys 'as is'
    keeperkeys = ['id', 'proposalURL', 'fundURL', 'imageURL', 'retinaImageURL', 'thumbImageURL', 'fulfillmentTrailer', 'percentFunded', 'numDonors', 'costToComplete', 'studentLed', 'numStudents', 'professionalDevelopment', 'distanceLearningProject', 'totalPrice', 'freeShipping', 'teacherId', 'teacherName', 'teacherTypes', 'schoolTypes', 'schoolName', 'schoolUrl', 'city', 'zip', 'state', 'stateFullName', 'latitude', 'longitude', 'expirationDate', 'expirationTime', 'fundingStatus', 'fullyFundedDate', 'waitingForCheckPayment', 'modifiedDate','shortDescription']
    
    #these require more finagling
    moreprocess = ['matchingFund', 'gradeLevel', 'povertyType']
    
    combine=[]
    
    for prop in proplist:
        new_dict = {k: prop[k] for k in keeperkeys if k in prop}
        keepinfo = pd.DataFrame.from_dict(new_dict,orient='index').T
        
        if(not bool(prop['matchingFund'].get('matchingKey'))):
            keepinfo['matchingfund'] = 'no'
        else:
            keepinfo['matchingfund'] = 'no'
            
        keepinfo['grade_level'] = prop['gradeLevel']['name']
        keepinfo['poverty_level'] = prop['povertyType']['label']
        combine.append(keepinfo)
        
    summarydf = pd.concat(combine)
    summarydf['r_path']=requestpath*summarydf.shape[0]
    return(summarydf)

In [50]:
#site = 'https://newyork.craigslist.org/d/for-sale/search/sss'
site = 'https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50'

In [59]:
from bs4 import BeautifulSoup
url = 'http://archive.org/wayback/available?url='+site+'&timestamp='+str(year)+str(month).zfill(2)+str('01')
url = 'https://web.archive.org/web/20120926032156/http://www.donorschoose.org/donors/search.html?state=AK'

In [60]:
r = requests.get(url) #send request to specific html address

In [67]:
soup = BeautifulSoup(r.text, 'html.parser')

In [187]:
# loop through states and add their city href

all_waybacks = []

for eachproject in soup.find_all("div", {"class": "candy proposalItem incompleted"}):
    thislist = []
    
    #get proposalid
    ####################################
    proposalid = str(eachproject.find_all("input", {"name": "proposalid"}))
    proposalid = proposalid.split('value="',1)[1]
    proposalid = proposalid.split('"/')[0]
    
    #project title
    #################################
    projtitle = str(eachproject).split('<span id="resultTitle">',1)[1]
    projtitle = projtitle.split('</span>')[0]
    
    #pull short description
    ######################################
    short_descrip=((eachproject.find_all("div", {"class": "shortDesc"})[0]).find_all("h2", {"id":"resultDescription"}))[0].get_text().strip().lower()
    sep = '...'
    short_descrip = short_descrip.split(sep, 1)[0]
    
    #students need
    ##################################
    students_need = str(eachproject.find_all("div", {"class": "myStudentsNeed"}))
    students_need = re.sub('<[^>]+>', '', students_need).strip().lower()
    students_need = students_need.replace('[','')
    students_need = students_need.replace(']','')
    students_need = students_need.strip()
    
    #location
    ###########################
    p_location = str(eachproject.find_all("div", {"id": "resultLocationInfo"}))
    p_location = re.sub('<[^>]+>', '', p_location).strip().lower()
    p_location = p_location.replace('[','')
    p_location = p_location.replace(']','')
    
    #poverty and teacher
    ###################################
    basic_info = str(eachproject.find_all("a"))
    basic_info = re.sub('<[^>]+>', '', basic_info).strip().lower()
    basic_info = basic_info.split(",")
    
    #default teacher and poverty values, if conditions are never met below
    teacher = ''
    poverty = ''
    
    for b in basic_info:
        if 'mr.' in b:
            teacher = b.strip()
        if 'ms.' in b:
            teacher = b.strip()
        if 'mrs.' in b:
            teacher = b.strip()         
        if 'povert' in b:
            poverty = b.strip()
    
    ######################################
    thislist = thislist +[proposalid]+[projtitle]+[short_descrip]+[students_need]+[p_location]+[teacher]+[poverty]
    
    all_waybacks.append(thislist)

In [188]:
all_waybacks

[['817912',
  'WI Fast Plants in AK Pt.2',
  'our school serves k-12 students in rural ak. this is next to the bearing sea, where the shows "deadliest catch" and "flying wild ak" are filmed. the students face the harsh weather and lack of jobs',
  'my students need plant materials and soil to continue learning about plants and biology.',
  'hooper bay, ak',
  'mr. baldwin',
  'high poverty'],
 ['850872',
  'Assessment Tools to Check for Understanding',
  'as many of my students have a difficult time demonstrating their understanding with pencil and paper tests using dry erase boards gives them an alternate method to ask questions, show whether or not',
  'my students need dry erase boards and markers, magnetic nameplates and a timer to offer them the opportunity to demonstrate their understanding of the content being taught.',
  'anchorage, ak',
  'mrs. oldham',
  'high poverty'],
 ['864970',
  "Let's Cook! Using Measurement in PreK",
  'help us learn to measure and make great treats a

In [167]:
eachproject = soup.find_all("div", {"class": "candy proposalItem incompleted"})[0]
#print(eachproject)

In [172]:
import glob #pattern matching and expansion.

In [186]:
    #poverty
    basic_info = str(eachproject.find_all("a"))
    basic_info = re.sub('<[^>]+>', '', basic_info).strip().lower()
    basic_info = basic_info.split(",")
    
    for b in basic_info:
        if 'mr.' in b:
            teacher = b.strip()
        if 'ms.' in b:
            teacher = b.strip()
        if 'mrs.' in b:
            teacher = b.strip()         
        if 'povert' in b:
            poverty = b.strip()
#     p_location = p_location.replace('[','')
#     p_location = p_location.replace(']','')
#     students_need = students_need.strip()
    print(basic_info)
    print(teacher)
    print(poverty)

['[', ' mr. baldwin', ' favorite', ' wi fast plants in ak pt.2', ' favorite', ' more', ' high poverty', ' ', ' give', ' ', ' ', ' ', ' ', ' ', ' ', ' favorites', ' favorite', ' giving page]']
mr. baldwin
high poverty


In [183]:
basic_info[1]

' mr. baldwin'

In [111]:
    sd = ((soup.find_all("div", {"class": "shortDesc"})[0]).find_all("h2", {"id":"resultDescription"}))[0].get_text().strip().lower()
    sep = '...'
    short_desc = sd.split(sep, 1)[0]
    print(short_desc)

our school serves k-12 students in rural ak. this is next to the bearing sea, where the shows "deadliest catch" and "flying wild ak" are filmed. the students face the harsh weather and lack of jobs....
	
	
		
	more
our school serves k-12 students in rural ak. this is next to the bearing sea, where the shows "deadliest catch" and "flying wild ak" are filmed. the students face the harsh weather and lack of jobs


In [89]:
state_item=(soup.find_all("div", {"class": "shortDesc"})[0])
print(state_item)

<div class="shortDesc">
<h2 id="resultDescription">	
	
			
	
	
		
	
	Our School serves k-12 students in rural AK. This is next to the Bearing Sea, where the shows "Deadliest Catch" and "Flying Wild AK" are filmed. The students face the harsh weather and lack of jobs....
	
	
		
	<a href="https://web.archive.org/web/20120926032156/http://www.donorschoose.org/project/wi-fast-plants-in-ak-pt2/817912/?more=true" onclick="setIndexCookie(0,817912)" target="_top"><b>more</b></a>
</h2>
</div>


In [100]:
forks = state_item.find_all("h2", {"id":"resultDescription"})

In [104]:
forks

[<h2 id="resultDescription">	
 	
 			
 	
 	
 		
 	
 	Our School serves k-12 students in rural AK. This is next to the Bearing Sea, where the shows "Deadliest Catch" and "Flying Wild AK" are filmed. The students face the harsh weather and lack of jobs....
 	
 	
 		
 	<a href="https://web.archive.org/web/20120926032156/http://www.donorschoose.org/project/wi-fast-plants-in-ak-pt2/817912/?more=true" onclick="setIndexCookie(0,817912)" target="_top"><b>more</b></a>
 </h2>]

In [105]:
len(forks)

1

In [106]:
forks[0].get_text().strip().lower()

'our school serves k-12 students in rural ak. this is next to the bearing sea, where the shows "deadliest catch" and "flying wild ak" are filmed. the students face the harsh weather and lack of jobs....\n\t\n\t\n\t\t\n\tmore'

In [63]:
summary = [h.text for h in soup.findAll('p', attrs={'class': 'summary'})] + [h.text for h in soup.findAll('dd', attrs={'class': 'summary'})]


In [64]:
summary

[]

In [ ]:
 #Make the request
            try:
                r = requests.get(url) #send request to specific html address
            except:
                #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                continue #if we have an exception, *continue* will skip the rest of the iteration

            try:
                data_dict = json.loads(r.text) #load the text as a dictionary
            except: #catch *all* exceptions
                #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                continue #if we have an exception, *continue* will skip the rest of the iteration

            if data_dict["archived_snapshots"]!='': #if archived_snapshots is empty, this will be true, and we can skip the rest of the loop
                continue
                        
            dicts_on_this_page = list(data_dict.values())
            proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
            proposal_list = dicts_on_this_page[proposal_ind]

In [52]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
addme = 'state='
statelist = [addme + s for s in states]
statelist.append('')

subjectlist = ["",'subject5=-5','subject5=32','subject5=16','subject5=21','subject5=18','subject5=17','subject5=19','subject5=20',
               'subject2=-2','subject2=29','subject2=27','subject2=28','subject2=2',
              'subject3=-3','subject3=13','subject3=14','subject3=9','subject3=15',
               'subject4=-4','subject4=6','subject4=7','subject4=30','subject4=4','subject4=8',
               'subject1=-1','subject1=12','subject1=1','subject1=11']

showonlylist = ['','metroType=r','teacherNotFunded=true','moderateHighEconomicNeed=true','distanceLearning=only']


In [53]:
wayback_urls =[]

#loop through every state
for state in statelist:
    if state == '':
        combo_url = site
    else:
        combo_url = site+'&'+state
        
    for subject in subjectlist:
        if subject == '':
            new_combo_url = combo_url
        else:
            new_combo_url = combo_url+'&'+subject
            
        for showme in showonlylist:
            if showme == '':
                double_new_url = new_combo_url
            else:
                double_new_url = new_combo_url+'&'+showme
                
            wayback_urls.append(double_new_url)

In [54]:
print(len(wayback_urls))
shortway=[wayback_urls[0]]
print(shortway)


7540
['https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&state=AL']


In [ ]:
https://web.archive.org/web/20200404034311/https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&index=0

In [49]:
wayback_counter=0
prog_count = 0
iteration = 0
out_file = "/home/russell/Documents/DataScience/DonorsChoose/Data/Waybacks/pickle_"


#pbar = tqdm(total=len(wayback_urls))
for wayback in shortway:
#for wayback in wayback_urls:
    wayback_counter += 1
    for year in range(2010,2020+1):
        for month in range(1,12+1):
            url = 'http://archive.org/wayback/available?url='+wayback+'&timestamp='+str(year)+str(month).zfill(2)+str('01')
            
             #Make the request
            try:
                r = requests.get(url) #send request to specific html address
            except:
                #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                continue #if we have an exception, *continue* will skip the rest of the iteration

            try:
                data_dict = json.loads(r.text) #load the text as a dictionary
            except: #catch *all* exceptions
                #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                continue #if we have an exception, *continue* will skip the rest of the iteration

            if data_dict["archived_snapshots"]!='': #if archived_snapshots is empty, this will be true, and we can skip the rest of the loop
                continue
                        
            dicts_on_this_page = list(data_dict.values())
            proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
            proposal_list = dicts_on_this_page[proposal_ind]

            #if projects are returned with these criteria, len(proposal_list) will be > 0
            if len(proposal_list)>0:

                    outframe = proposal_process(proposal_list,url)
                    #print(outframe['teacherName'])
                    pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
                    pickle.dump(outframe, pickle_out)
                    pickle_out.close()
                    iteration += 1
                    prog_count += 50
                    more_to_get = True
                    indexup = 0
                    time.sleep(np.random.rand()*5)
                    
                    while more_to_get:
                        indexup +=50
                        new_wayback = wayback+'&index='+str(indexup)
                        url = 'http://archive.org/wayback/available?url='+new_wayback+'&timestamp='+str(year)+str(month).zfill(2)+str('01')
                        
                        try:
                            r = requests.get(url) #send request to specific html address
                        except:
                            #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                            more_to_get = False
                            continue #if we have an exception, *continue* will skip the rest of the iteration

                        try:
                            data_dict = json.loads(r.text) #load the text as a dictionary
                        except: #catch *all* exceptions
                            #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                            more_to_get = False
                            continue #if we have an exception, *continue* will skip the rest of the iteration

                        if data_dict["archived_snapshots"]!='': #if archived_snapshots is empty, this will be true, and we can skip the rest of the loop
                            continue    
                            
                        dicts_on_this_page = list(data_dict.values())
                        proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
                        proposal_list = dicts_on_this_page[proposal_ind]

                        #if projects are returned with these criteria, len(proposal_list) will be > 0
                        if len(proposal_list)>0:
                            outframe = proposal_process(proposal_list,page_to_use)
                            #print(outframe['teacherName'])
                            pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
                            pickle.dump(outframe, pickle_out)
                            pickle_out.close()
                            iteration += 1
                            prog_count += 50
                            time.sleep(np.random.rand()*5)
                        else:
                            more_to_get = False
            
    #pbar.update(1)
    #print("Current progress:",np.round(iteration/(len(wayback_urls))),"%")
    update_progress(wayback_counter / len(wayback_urls))        
            
#pbar.close()

Progress: [--------------------] 0.0%


In [29]:
print(url)

http://archive.org/wayback/available?url=https://newyork.craigslist.org/d/for-sale/search/sss&timestamp=20141201


In [ ]:
url = 'http://archive.org/wayback/available?url='+wayback+'&timestamp='+str(year)+str(month).zfill(2)+str('01')
            
             #Make the request
            try:
                r = requests.get(url) #send request to specific html address
            except:
                #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                continue #if we have an exception, *continue* will skip the rest of the iteration

            try:
                data_dict = json.loads(r.text) #load the text as a dictionary
            except: #catch *all* exceptions
                #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                continue #if we have an exception, *continue* will skip the rest of the iteration

            if data_dict["archived_snapshots"]!='': #if archived_snapshots is empty, this will be true, and we can skip the rest of the loop
                continue
                        
            dicts_on_this_page = list(data_dict.values())
            proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
            proposal_list = dicts_on_this_page[proposal_ind]

            #if projects are returned with these criteria, len(proposal_list) will be > 0
            if len(proposal_list)>0:

                    outframe = proposal_process(proposal_list,url)
                    #print(outframe['teacherName'])
                    pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
                    pickle.dump(outframe, pickle_out)
                    pickle_out.close()
                    iteration += 1
                    prog_count += 50
                    more_to_get = True
                    indexup = 0
                    time.sleep(np.random.rand()*5)
                    
                    while more_to_get:
                        indexup +=50
                        new_wayback = wayback+'&index='+str(indexup)
                        url = 'http://archive.org/wayback/available?url='+new_wayback+'&timestamp='+str(year)+str(month).zfill(2)+str('01')
                        
                        try:
                            r = requests.get(url) #send request to specific html address
                        except:
                            #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                            more_to_get = False
                            continue #if we have an exception, *continue* will skip the rest of the iteration

                        try:
                            data_dict = json.loads(r.text) #load the text as a dictionary
                        except: #catch *all* exceptions
                            #tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
                            more_to_get = False
                            continue #if we have an exception, *continue* will skip the rest of the iteration

                        if data_dict["archived_snapshots"]!='': #if archived_snapshots is empty, this will be true, and we can skip the rest of the loop
                            continue    
                            
                        dicts_on_this_page = list(data_dict.values())
                        proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
                        proposal_list = dicts_on_this_page[proposal_ind]

                        #if projects are returned with these criteria, len(proposal_list) will be > 0
                        if len(proposal_list)>0:
                            outframe = proposal_process(proposal_list,page_to_use)
                            #print(outframe['teacherName'])
                            pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
                            pickle.dump(outframe, pickle_out)
                            pickle_out.close()
                            iteration += 1
                            prog_count += 50
                            time.sleep(np.random.rand()*5)
                        else:
                            more_to_get = False

In [30]:
r = requests.get(url)
data = json.loads(r.text)
newurl = data['archived_snapshots']['closest']['url']

In [31]:
p = requests.get(newurl)
soup = BeautifulSoup(p.text)
summary = [h.text for h in soup.findAll('p', attrs={'class': 'summary'})] + [h.text for h in soup.findAll('dd', attrs={'class': 'summary'})]


In [24]:
type(soup)

bs4.BeautifulSoup

In [32]:
soup

<html><body><p>﻿<!DOCTYPE html>

</p><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app23.us.archive.org';v.server_ms=562;archive_analytics.send_pageview({});});</script><script charset="utf-8" src="/_static/js/playback.bundle.js?v=R56vIgLu" type="text/javascript"></script>
<script charset="utf-8" src="/_static/js/wombat.js?v=nWgLpXRB" type="text/javascript"></script>
<script type="text/javascript">
if (window._WBWombatInit) {
  wbinfo = {}
  wbinfo.url = "https://newyork.craigslist.org/d/for-sale/search/sss";
  wbinfo.timestamp = "20180720024621";
  wbinfo.request_ts = "20180720024621";
  wbinfo.prefix = "http://web.archive.org/web/";
  wbinfo.mod = "if_";
  wbinfo.is_framed = false;
  wbinfo.is_live = false;
  wbinfo.coll = "web";
  wbinfo.proxy_magic = "";
  wbinfo.static_prefix = "/_stati

In [6]:
site = 'https://newyork.craigslist.org/'
for year in range(2010,2014+1):
	for month in range(1,12+1):
		url = 'http://archive.org/wayback/available?url='+site+'&timestamp='+str(year)+str(month).zfill(2)+str('01')
		r = requests.get(url)
		data = json.loads(r.text)
		newurl = data['archived_snapshots']['closest']['url']
		p = requests.get(newurl)
		soup = BeautifulSoup(p.text)
		summary = [h.text for h in soup.findAll('p', attrs={'class': 'summary'})] + [h.text for h in soup.findAll('dd', attrs={'class': 'summary'})]
		matching = [s for s in summary if 'Obama' in s]
		for item in matching:
			print (month,year,newurl,item.replace('\r\n','').encode('utf-8'))

2 2010 http://web.archive.org/web/20100201073434/http://www.nytimes.com:80/ b'\nThe additional tax cuts and public works spending that President Obama has proposed would bring this year\xe2\x80\x99s deficit to nearly $1.6 trillion.    '
3 2010 http://web.archive.org/web/20100301085559/http://www.nytimes.com/ b'\nAides to President Obama say he will permanently reduce America\xe2\x80\x99s arsenal by thousands of weapons.    '
5 2010 http://web.archive.org/web/20100501021016/http://www.nytimes.com:80/ b'\nMichelle Obama in three outfits created by the designer Maria Pinto, who said in an interview that she closed her business because of the economy and start-up mistakes.        '
6 2010 http://web.archive.org/web/20100531225053/http://www.nytimes.com/ b'\nPrime Minister Benjamin Netanyahu of Israel canceled his plans for meeting with President Obama after a raid on ships carrying supplies for Gaza left at least 10 people dead.    '
6 2010 http://web.archive.org/web/20100531225053/http://

9 2013 http://web.archive.org/web/20130901005130/http://www.nytimes.com/ b'\nPresident Obama paused his march to battle by asking lawmakers, who are not due to return to town for more than a week, to give him authorization before he launches a limited military strike against Syria.    '
9 2013 http://web.archive.org/web/20130901005130/http://www.nytimes.com/ b'\nWhen President Obama said he would seek Congressional approval for military strikes, Syrians were shocked, angry and relieved. Above, a family crossed into Turkey.        '
9 2013 http://web.archive.org/web/20130901005130/http://www.nytimes.com/ b'\nThe documents show that the United States conducted 231 cyberoperations in 2011, even while the Obama administration protested other countries\xe2\x80\x99 attacks on American computer networks.    '
10 2013 http://web.archive.org/web/20131001014236/http://www.nytimes.com/ b'\nThe federal government teetered on the brink of its first shutdown in nearly two decades after last-minute m